In [1]:
'''Zadanie 1'''
import pandas as pd
import numpy as np
import dask.dataframe as dd

df = pd.read_csv('zamowienia.csv')

df.loc[10:12, 'kolumna1'] = np.nan
df.loc[15:17, 'kolumna2'] = np.nan
df.loc[20:22, 'kolumna3'] = np.nan

df.to_csv('zamowienia_missing.csv', index=False)

ddf = dd.read_csv('zamowienia_missing.csv')

print("Typy danych w ramce Dask:")
print(ddf.dtypes)

result = ddf['kolumna1'].sum().compute()
print("Suma wartości w kolumnie 'kolumna1':", result)

ddf_sample1 = dd.read_csv('zamowienia_missing.csv', sample=1000)
ddf_sample2 = dd.read_csv('zamowienia_missing.csv', sample=5000)

print("Typy danych w ramce Dask z sample=1000:")
print(ddf_sample1.dtypes)

print("Typy danych w ramce Dask z sample=5000:")
print(ddf_sample2.dtypes)

Typy danych w ramce Dask:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    string[pyarrow]
kolumna1                                                      float64
kolumna2                                                      float64
kolumna3                                                      float64
dtype: object
Suma wartości w kolumnie 'kolumna1': 0.0
Typy danych w ramce Dask z sample=1000:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    string[pyarrow]
kolumna1                                                      float64
kolumna2                                                      float64
kolumna3                                                      float64
dtype: object
Typy danych w ramce Dask z sample=5000:
Kraj;Sprzedawca;Data zamowienia;idZamowienia;Utarg    string[pyarrow]
kolumna1                                                      float64
kolumna2                                                      float64
kolumna3                                               

In [2]:
'''Zadanie 2'''
from dask.distributed import Client
client = Client()
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [3]:
'''Zadanie 3'''
import dask.dataframe as dd
from dask.distributed import Client
import time

client = Client(memory_limit='16GB', n_workers=6, threads_per_worker=1)

print(client)

file_list = [f"{str(i).zfill(4)}.parquet" for i in range(15)]

start_time = time.time()

df = dd.read_parquet(file_list, engine='pyarrow')

print(df.head())

end_time = time.time()
print(f"Czas załadowania danych: {end_time - start_time:.2f} sekundy")

        sid  sid_profile      post_id  profile_id                 date  \
0  28370919      3496776  BXdjjUlgcgq  2237947779  2017-08-06 20:06:57   
1  13623950      3496776  BeyPed5hKj9  2237947779  2018-02-04 19:35:20   
2  28370905      3496776  Bunhd1DFVAG  2237947779  2019-03-05 08:03:11   
3  28370907      3496776  Bppi85gliQK  2237947779  2018-11-01 20:17:41   
4  32170690      3496776  BuDfIyslzfw  2237947779  2019-02-19 08:10:11   

   post_type                                        description  likes  \
0          2  Wreckloose! Deevalley bike park laps on the @i...     80   
1          1  The dirty south was prime today. Top day with ...     86   
2          1  Tech Tuesday. Been flat out on the tools. Got ...    168   
3          1  On the tools, my favourite wheel builds @stans...    102   
4          1  Solid effort on the bar turn.\nFully turned.\n...    145   

   comments   username                                                bio  \
0         0  andylund_  Professio

In [10]:
'''Zadanie 4'''
import dask.dataframe as dd
from dask.distributed import Client
import time

client = Client(memory_limit='16GB', n_workers=6, threads_per_worker=1)

file_list = [f"{str(i).zfill(4)}.parquet" for i in range(15)]

df = dd.read_parquet(file_list, engine='pyarrow')

start_time = time.time()

top_users = df.groupby('username')['likes'].sum().nlargest(10).compute()

end_time = time.time()
print("Top 10 użytkowników z najwyższą liczbą like'ów:")
print(top_users)
print(f"Czas operacji: {end_time - start_time:.2f} sekundy")

start_time = time.time()

df['date'] = dd.to_datetime(df['date'])
filtered_df = df[(df['date'] >= '2019-01-01') & (df['date'] < '2019-07-01')].compute()

end_time = time.time()
print(f"Liczba rekordów za pierwsze półrocze 2019 roku: {len(filtered_df)}")
print(f"Czas operacji: {end_time - start_time:.2f} sekundy")

print(filtered_df.head())

Top 10 użytkowników z najwyższą liczbą like'ów:
username
lilireinhart     40375292
instagram        29864166
jamescharles     26067462
lizakoshy        19217644
elle             17494288
433              16457870
amandacerny      15019135
saraalikhan95    14199489
zkdlin           13799100
akshaykumar      13352324
Name: likes, dtype: int64
Czas operacji: 3.51 sekundy


In [8]:
'''Zadanie 5'''
import dask.dataframe as dd
from dask.distributed import Client
import time

client = Client(memory_limit='16GB', n_workers=6, threads_per_worker=1)

file_list = [f"{str(i).zfill(4)}.parquet" for i in range(15)]

dtypes = {
    'username': 'string',
    'likes': 'int',
    'date': 'datetime64[ns]'
}

start_time = time.time()
df = dd.read_parquet(file_list, engine='pyarrow', dtype=dtypes)
end_time = time.time()
print(f"Czas ładowania danych z określonymi typami: {end_time - start_time:.2f} sekundy")

start_time = time.time()
top_users = df.groupby('username')['likes'].sum().nlargest(10).compute()
end_time = time.time()
print("Top 10 użytkowników z najwyższą liczbą like'ów:")
print(top_users)
print(f"Czas operacji: {end_time - start_time:.2f} sekundy")

start_time = time.time()
filtered_df = df[(df['date'] >= '2019-01-01') & (df['date'] < '2019-07-01')].compute()
end_time = time.time()
print(f"Liczba rekordów za pierwsze półrocze 2019 roku: {len(filtered_df)}")
print(f"Czas operacji: {end_time - start_time:.2f} sekundy")

print(filtered_df.head())

Czas ładowania danych z określonymi typami: 0.05 sekundy
Top 10 użytkowników z najwyższą liczbą like'ów:
username
lilireinhart     40375292
instagram        29864166
jamescharles     26067462
lizakoshy        19217644
elle             17494288
433              16457870
amandacerny      15019135
saraalikhan95    14199489
zkdlin           13799100
akshaykumar      13352324
Name: likes, dtype: int64
Czas operacji: 3.36 sekundy
Liczba rekordów za pierwsze półrocze 2019 roku: 10890703
Czas operacji: 67.12 sekundy
        sid  sid_profile      post_id  profile_id                 date  \
2  28370905      3496776  Bunhd1DFVAG  2237947779  2019-03-05 08:03:11   
4  32170690      3496776  BuDfIyslzfw  2237947779  2019-02-19 08:10:11   
5  14315358      3496776  BxJsMDpA2yH  2237947779  2019-05-07 08:33:51   
6   8304346      3496776  Bt5LFpZlm3z  2237947779  2019-02-15 08:02:35   
7  14315346      3496776  BxZIzaQhS-o  2237947779  2019-05-13 08:32:30   

   post_type                             

In [7]:
'''Zadanie 6'''
import dask.array as da
import numpy as np
import time

np_array = np.random.random(size=(10000, 10000))

def measure_time(chunks):
    darr = da.from_array(np_array, chunks=chunks)
    times = []
    for _ in range(10):
        start_time = time.time()
        mean_value = darr.mean().compute()
        end_time = time.time()
        times.append(end_time - start_time)
    return np.mean(times), mean_value

chunk_sizes = [(1000, 1000), (2000, 2000), (5000, 5000), (10000, 10000)]
results = {}

for chunks in chunk_sizes:
    avg_time, mean_value = measure_time(chunks)
    results[chunks] = (avg_time, mean_value)
    print(f"Chunks: {chunks}, Average Time: {avg_time:.4f} seconds, Mean: {mean_value}")

print("\nWnioski:")
for chunks, (avg_time, mean_value) in results.items():
    print(f"Podział na chunki {chunks} - średni czas: {avg_time:.4f} sekundy")

Wnioski:
Podział na chunki (1000, 1000) - średni czas: 4.4321 sekundy
Podział na chunki (2000, 2000) - średni czas: 3.9757 sekundy
Podział na chunki (5000, 5000) - średni czas: 3.7703 sekundy
Podział na chunki (10000, 10000) - średni czas: 2.4939 sekundy
